# Chess Engine with TensorFlow

## Dataset

In [1]:
import os
# inspired by Github user Skripkon 
# https://github.com/Skripkon/chess-engine/blob/main/engines/tensorflow/train_and_predict.ipynb
# used to train models from scratch
# took around 3 hours for 20000 games

# get the game files
files = [file for file in os.listdir("simulated_games_filtered_PGN") if file.endswith(".pgn")]

In [2]:
from chess import pgn

# load the games
def load_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
            
    return games

In [3]:
from tqdm import tqdm
# write all the games together 

games = []
for file in tqdm(files):
    games.extend(load_pgn(f"simulated_games_filtered_PGN/{file}"))

100%|██████████| 11/11 [00:12<00:00,  1.16s/it]


In [4]:
len(games) # check how many 

10085

## Build & train a neural network

In [5]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from chess import Board
import tensorflow as tf

In [6]:
#translating the board into a matrix for the predition process 
def board_to_matrix(board: Board):
    matrix = np.zeros((8, 8, 12))
    piece_map = board.piece_map()
    for square, piece in piece_map.items():
        row, col = divmod(square, 8)
        piece_type = piece.piece_type - 1
        piece_color = 0 if piece.color else 6
        matrix[row, col, piece_type + piece_color] = 1
    return matrix

# create the inputs based off game position and next move 
def create_input_for_nn(games):
    X = []
    y = []
    for game in games:
        board = game.board()
        for move in game.mainline_moves():
            X.append(board_to_matrix(board))
            y.append(move.uci())
            board.push(move)
    return X, y

# encode all the moves
def encode_moves(moves):
    move_to_int = {move: idx for idx, move in enumerate(set(moves))}
    return [move_to_int[move] for move in moves], move_to_int

In [7]:
# create the training data
X, y = create_input_for_nn(games)
y, move_to_int = encode_moves(y)
y = tf.keras.utils.to_categorical(y, num_classes=len(move_to_int))
X = np.array(X)

In [ ]:

# train the model
model = tf.keras.models.Sequential([
    #Changed activation function to swish
    tf.keras.layers.Conv2D(64, (3, 3), activation='gelu', input_shape=(8, 8, 12)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='gelu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='gelu'),
    
    #Keeping softmax activation for control
    tf.keras.layers.Dense(len(move_to_int), activation='softmax')
])
model.compile(optimizer = tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X, y, epochs=50, validation_split=0.1, batch_size=64)
# save the model
model.save("models/simulated_filtered_model(gelu)/SSMF_50EPOCHS.keras")

import pickle
# save the encoding
with open("models/simulated_filtered_model(gelu)/move_to_int.pkl", "wb") as f:
    pickle.dump(move_to_int, f)
int_to_move = {v: k for k, v in move_to_int.items()}
with open("models/simulated_filtered_model(gelu)/int_to_move.pkl", "wb") as f:
    pickle.dump(int_to_move, f)
# configuration 
config = {
    "epochs": 50,
    "batch_size": 64,
    "validation_split": 0.1,
    "optimizer": "Adam",
    "input_shape": (8, 8, 12),
}
# save the configurations
with open("models/simulated_filtered_model(gelu)/train_config.json", "w") as f:
    import json
    json.dump(config, f, indent=4)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 6, 6, 64)          6976      
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 1945)              499865    
                                                                 
Total params: 1105241 (4.22 MB)
Trainable params: 1105241 (4.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
15743/1

KeyboardInterrupt: 

: 